In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
# Define the silver location
genre_silverLocation = "/mnt/datalake/silver/cleaned_genre"

# Table name
genre_bronzeTable = "bronze.genre"
genre_silverTable = "silver.genre"

In [0]:
df_genre = spark.read.table('bronze.genre')
df_movie_genre = spark.read.table('bronze.movie_to_genre')

In [0]:
df_genre.show(5)

+---+--------------------+--------------------+----------+---------+
| id|          created_at|          updated_at|deleted_at|     name|
+---+--------------------+--------------------+----------+---------+
|  1|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|   Action|
|  2|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Adult|
|  3|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|Adventure|
|  4|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|Animation|
|  5|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|Biography|
+---+--------------------+--------------------+----------+---------+
only showing top 5 rows



In [0]:
df_movie_genre.show(5)

+-------+-------+
|movieId|genreId|
+-------+-------+
|      1|      6|
|      1|     11|
|      1|     21|
|      2|      7|
|      2|      9|
+-------+-------+
only showing top 5 rows



In [0]:
df_genre = df_genre.dropDuplicates(['id'])
df_movie_genre = df_movie_genre.dropDuplicates()

#### Rename keywordId -> id, name -> keywords

In [0]:
df_movie_genre = df_movie_genre.withColumnRenamed('genreId', 'id')
df_genre = df_genre.withColumnRenamed('name', 'genres')

#### Left join

In [0]:
df_new = df_movie_genre.join(df_genre , on=['id'] , how = 'left')

In [0]:
df_new.show()

+---+-------+--------------------+--------------------+----------+---------+
| id|movieId|          created_at|          updated_at|deleted_at|   genres|
+---+-------+--------------------+--------------------+----------+---------+
|  6|      1|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|   Comedy|
| 11|      1|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|  Fantasy|
| 21|      1|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|  Romance|
|  7|      2|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Crime|
|  9|      2|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Drama|
|  9|      3|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Drama|
| 15|      3|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|   Horror|
|  7|      4|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Crime|
|  9|      4|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|    Drama|
| 15|      5|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|   Horror|

In [0]:
df_new = df_new.select('movieId', 'genres')

In [0]:
df_new.sort(df_new['movieId'].asc()).show()
df_new.count()

+-------+-------+
|movieId| genres|
+-------+-------+
|      1| Comedy|
|      1|Fantasy|
|      1|Romance|
|      2|  Crime|
|      2|  Drama|
+-------+-------+
only showing top 5 rows



255181

### drop duplicates

In [0]:
df_new = df_new.dropDuplicates(subset=['movieId', 'genres'])
df_new.count()


255181

In [0]:
from pyspark.sql.functions import collect_list
df_new = df_new.groupBy("movieId").agg(collect_list("genres").alias("genres"))
df_new.show()

+-------+--------------------+
|movieId|              genres|
+-------+--------------------+
|      1|[Comedy, Fantasy,...|
|      3|     [Horror, Drama]|
|      5|   [Mystery, Horror]|
|      6|[War, Western, Hi...|
|     12|     [Drama, Comedy]|
|     13|             [Drama]|
|     15|            [Comedy]|
|     16|             [Drama]|
|     19|[Mystery, Drama, ...|
|     20|             [Drama]|
|     22|[Thriller, Horror...|
|     26|[Crime, Drama, Sp...|
|     27|[War, Drama, Adve...|
|     28| [Family, Adventure]|
|     31|[Drama, Romance, ...|
|     34|[Romance, Sci-Fi,...|
|     40|    [Drama, Romance]|
|     41|  [Biography, Drama]|
|     43|[Romance, Western...|
|     44|[Comedy, Family, ...|
+-------+--------------------+
only showing top 20 rows



#### Save table

In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:
df_new.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(genre_silverLocation)
    

In [0]:
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {genre_silverTable} USING delta LOCATION '/mnt/datalake/silver/cleaned_genre'")

DataFrame[]

In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
genre_DataDelta = DeltaTable.forName(spark, genre_silverTable)

if genre_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	genre_DataDelta.optimize()
	genre_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.genre;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2023-12-13T11:20:12Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(1379132818750927),1122-230829-ksqo7xkx,4,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
4,2023-12-13T11:20:08Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(1379132818750927),1122-230829-ksqo7xkx,3,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
3,2023-12-13T11:19:51Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1379132818750927),1122-230829-ksqo7xkx,2,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 108824, numOutputBytes -> 631169)",null,Databricks-Runtime/13.3.x-scala2.12
2,2023-12-13T11:16:28Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1379132818750927),1122-230829-ksqo7xkx,1,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 108824, numOutputBytes -> 631169)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-13T11:15:34Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1379132818750927),1122-230829-ksqo7xkx,0,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 108824, numOutputBytes -> 631169)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-13T11:13:40Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1379132818750927),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 108824, numOutputBytes -> 631169)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:

%sql
select * from silver.genre

movieId,genres
1,"List(Comedy, Fantasy, Romance)"
3,"List(Horror, Drama)"
5,"List(Mystery, Horror)"
6,"List(War, Western, History, Drama, Adventure)"
12,"List(Drama, Comedy)"
13,List(Drama)
15,List(Comedy)
16,List(Drama)
19,"List(Mystery, Drama, Thriller)"
20,List(Drama)
